In [2]:
import numpy as np
import cv2
import scipy.io
import os
from numpy.linalg import norm
#from matplotlib import pyplot as plt
from numpy.linalg import det
from numpy.linalg import inv
from scipy.linalg import rq
from numpy.linalg import svd
import matplotlib.pyplot as plt
!pip install -U matplotlib
!pip install pytransform3d
!pip install ipympl
!pip install utils
%matplotlib widget
import matplotlib.pyplot as plt
from utils import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.2 MB 9.7 MB/s 
     |████████████████████████████████| 944 kB 17.5 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 4.3 MB/s 
  Created wheel for pytransform3d: filename=pytransform3d-1.14.0-py3-none-any.whl size=90191 sha256=f9a1d2110a88c72b3b937e148dbe0094415a994cbe2bfa606cf8b2e74ac3906d
  Stored in directory: /root/.cache/pip/wheels/d8/76/97/bf646631f5136cbf107a6067ed65cabf4ed5af679209329b54
Successfully built pytransform3d
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 508 kB 7.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


in order to calibrate the camera we need to calculate it's internal parameters 


---
p= K [R.t] P


Given corresponding 2d points in the image and 3d coordinates from the satellite image with known extrinsics, estimate the camera intrinsics

In [3]:
data_part1 = scipy.io.loadmat('pt_corres.mat')
cam_pts_3D = data_part1['cam_pts_3D']         # Load the 3d points
pts_2D = data_part1['pts_2D']                 # Load the corresponding 2d points

print(pts_2D.shape)
print(cam_pts_3D.shape)

#Obtain the transpose of two matrices
pts_2DT=np.transpose(pts_2D)
cam_pts_3DT=np.transpose(cam_pts_3D)

one=np.ones((7,1))

#Matrix AT is composed of three columns with the first column being the x values of the image points, 
#second column being the y values of the image points and the third column being zeros 

A = np.hstack((pts_2DT,one))
AT=np.transpose(A)

#Compose matrix BT such that 1st column is X/Z, 2nd column is Y/Z, 3rd column is one
x1=np.empty((7,1))
x2=np.empty((7,1))
for i in range(0,7):
  x1[i]=cam_pts_3DT[i][0]/cam_pts_3DT[i][2]
  x2[i]=cam_pts_3DT[i][1]/cam_pts_3DT[i][2]

B=np.hstack((x1,x2,one))
BT=np.transpose(B)
Binv = np.linalg.pinv(BT)
print("K", np.dot(AT,Binv))

(2, 7)
(3, 7)
K [[ 3.39842152e+03  2.60577300e+01  8.54342104e+02]
 [ 2.85229632e+01  3.24882954e+03  6.22845807e+02]
 [-8.88178420e-16  3.55271368e-15  1.00000000e+00]]


#Functions Used

In [4]:
def calibrate(x,X):
  '''
  This function computes camera projection matrix from 3D scene points
  and corresponding 2D image points with the Direct Linear Transformation (DLT).
  
  Usage:
  P = calibrate(x, X)
  
  Input:
  x: 2xn image points
  X: 3xn scene points

  Output:
  P: 3x4 camera projection matrix
  
  '''
  
  #The svd function from Numpy returns U, Sigma and V_transpose
  
  
  xt=np.transpose(x)
  Xt=np.transpose(X)
    
  Xt=np.hstack((Xt,np.ones((7,1))))
  print(Xt.shape)
  zero4=np.array((0,0,0,0))
  
  M=np.array((56,12))
  for i in range(0,7):
    A=np.hstack((zero4,-Xt[i],xt[i][1]*Xt[i]))
    B=np.hstack((Xt[i],zero4,-xt[i][0]*Xt[i]))
    A=np.reshape(A,(1,12))
    B=np.reshape(B,(1,12))
    if i == 0:
      M=np.vstack((A,B))
     
    else:
      M=np.vstack((M,A,B))
      
  u, s, vtranspose = np.linalg.svd(M)
  v=np.transpose(vtranspose)
  p= v[:,11]
  
  
  P=p.reshape((3,4))
  return P

In [5]:
def P_to_KRt(P):
  '''
  This function computes the decomposition of the projection matrix into intrinsic parameters, K, and extrinsic parameters Q (the rotation matrix) and t (the translation vector)
  
  Usage:
  K, Q, t = P_to_KRt(P)
  
  Input: 
  P: 3x4 projection matrix
  
  Outputs:
  K: 3x3 camera intrinsics
  Q: 3x3 rotation matrix (extrinsics)
  t: 3x1 translation vector(extrinsics)
  
  '''
  
  M = P[0:3,0:3]
  
  R, Q = rq(M)
    
  K = R/float(R[2,2])
  
  if K[0,0] < 0:
    K[:,0] = -1*K[:,0]
    Q[0,:] = -1*Q[0,:]
    
  if K[1,1] < 0:
    K[:,1] = -1*K[:,1]
    Q[1,:] = -1*Q[1,:]
  
  if det(Q) < 0:
    print ('Warning: Determinant of the supposed rotation matrix is -1')
  
  P_3_3 = np.dot(K,Q)
  
  P_proper_scale = (P_3_3[0,0]*P)/float(P[0,0])
  
  t = np.dot(inv(K), P_proper_scale[:,3])
  
  return K, Q, t


In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
#plt.style.use('seaborn-white')
import numpy as np

#Load the data

pts_3d = scipy.io.loadmat('3D_pts.mat')['pts_3d'] # 3D points in the world-coordinate system with choosing the origin center as approupriat
pts_2d = scipy.io.loadmat('2D_pts.mat')['pts_2d'] # The corresponding 2D points on the image testImg.jpg


  
P = calibrate(pts_2d, pts_3d)     

print ('P = ', P) 

# use P_to_KRt to decompose P into intrinsics (K) and extrinsics (R and t)
[K, R, t] = P_to_KRt(P)

print ('K = ', K)
print ('R = ', R)
print ('t = ', t)
Xt=np.transpose(pts_3d)
Xt=np.hstack((Xt,np.ones((7,1)))) # for 7 points
X_=np.transpose(Xt)

#averaged over the 7 points given
xnew=np.dot(P,X_)
xnewt=np.transpose(xnew)
x1=np.empty((7,1))
x2=np.empty((7,1))
for i in range(0,7):
  x1[i]=xnewt[i][0]/xnewt[i][2]
  x2[i]=xnewt[i][1]/xnewt[i][2]

x_proj=np.hstack((x1,x2))
xt=np.transpose(pts_2d)
sumerr=0
for i in range(0,7):
  sumerr+=np.square(xt[i][0]-x_proj[i][0])+np.square(xt[i][1]-x_proj[i][1])
reproj_err=np.sqrt(sumerr)/7
print("Reprojection Error", reproj_err)




# Display the given 2D points and the reprojected 2D points on the test image provided
fig, ax = plt.subplots(1)
img=cv2.imread("testImage.png")
xorg=xt[:,0]
yorg=xt[:,1]
xproj=x_proj[:,0]
yproj=x_proj[:,1]
for xx,yy in zip(xorg,yorg):
    circ = plt.Circle((xx,yy),20,color='r')
    ax.add_patch(circ)
    
for xx,yy in zip(xproj,yproj):
    circ = plt.Circle((xx,yy),10,color='g')
    ax.add_patch(circ)
    
ax.imshow(img)    


(7, 4)
P =  [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in true_divide


LinAlgError: ignored

In [ ]:
mat = scipy.io.loadmat('pt_corres.mat')
print(mat)